
### 1. Instalação de Dependências e Hiperparametros

In [16]:
# Instalação das dependências
!pip install transformers datasets evaluate sacrebleu pandas openpyxl torch peft sentencepiece

import warnings
warnings.filterwarnings('ignore')

import os
import json
import torch
import pandas as pd
import numpy as np
from pathlib import Path

from transformers import (
    MBart50TokenizerFast,
    MBartForConditionalGeneration,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)
from datasets import Dataset, DatasetDict

import evaluate

from peft import LoraConfig, get_peft_model, TaskType

preferred_device = "cuda"
device = torch.device(preferred_device if torch.cuda.is_available() else "cpu")



In [5]:
# Definir diretório raiz do projeto
PROJECT_ROOT_DIR = "/home/biel/Documentos/projetos-de-software/ep2/EP2-mac0508"

# Se estiver usando colab use 
# PROJECT_ROOT_DIR = "/content/drive/MyDrive/EP2-mac0508"

In [7]:
CONFIG = {

    "model_checkpoint": "facebook/mbart-large-50-many-to-many-mmt",
    "nllb_checkpoint": "facebook/nllb-200-distilled-600M",

    "max_input_length": 64,
    "max_target_length": 64,

    "learning_rate": 5e-4,
    "batch_size": 8,
    "num_epochs": 6,
    "weight_decay": 0.01,
    "warmup_ratio": 0.05,

    "early_stopping_patience": 3,

    "lora_r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,

    "project_root_dir": PROJECT_ROOT_DIR,
    "data_path": os.path.join(PROJECT_ROOT_DIR, "data.xlsx"),
    "processed_data_dir": os.path.join(PROJECT_ROOT_DIR, "processed_data"),
    "results_dir": os.path.join(PROJECT_ROOT_DIR, "results"),
    "models_dir": os.path.join(PROJECT_ROOT_DIR, "models"),

    "seed": 42
}

os.makedirs(CONFIG["results_dir"], exist_ok=True)
os.makedirs(CONFIG["models_dir"], exist_ok=True)

## 2. Carregamento e Preparação dos Dados

### 2.1 Leitura do Corpus

corpus Português/TupiAntigo em `data.xlsx`

**Observacao**: Preservei acentos, diacríticos e grafia histórica do Tupi Antigo.

In [8]:
df = pd.read_excel(CONFIG["data_path"])

df.columns = df.columns.str.replace('PortuguÊs', 'Português', regex=False)

print(f"Corpus carregado: {len(df)} pares de frases")
print(f"\nColunas: {list(df.columns)}")
print(f"\nPrimeiras 5 linhas:")
df.head()

Corpus carregado: 7097 pares de frases

Colunas: ['Português', 'Tupi Antigo']

Primeiras 5 linhas:


,Português,Tupi Antigo
0,Aparei as pontas deles,Aîapyr-etab
1,Doravante assim procedo,Ko'yré emonã aîkó
2,As doenças da alma do homem com ele saram bem,Abá 'anga mara'ara i pupé opûeîrá-katu
3,"É discreta, falando aos homens",I kunusãî abá supé onhe'enga
4,"Para se vingar de seu cão que cria, um homem n...","O eîmbaba îagûara resé oîepyka, abá n'oîmomba'..."


In [9]:
df['len_pt'] = df['Português'].astype(str).str.len()
df['len_ta'] = df['Tupi Antigo'].astype(str).str.len()
print(df[['len_pt', 'len_ta']].describe())

            len_pt       len_ta
count  7097.000000  7097.000000
mean     35.728054    26.222911
std      21.140072    17.144765
min       3.000000     2.000000
25%      20.000000    13.000000
50%      31.000000    21.000000
75%      47.000000    36.000000
max     187.000000   136.000000


### 2.2 Limpeza e Normalização

Realizei limpeza dos dados conforme orientação do professor:

#### Português:
1. **Remoção de expressões entre parênteses**: O corpus contém anotações explicativas entre parênteses que não aparecem na tradução Tupi
2. **Remoção de espaços extras** (início, fim, múltiplos espaços)
3. **Remoção de caracteres invisíveis**

#### Texto em Tupi Antigo:
1. **Apenas limpeza básica** - preservei acentos, diacríticos e grafia histórica
2. **NÃO removido**: acentos, diacríticos, maiúsculas/minúsculas

In [10]:
import re

def clean_text(text):

    if pd.isna(text):
        return ""
    
    text = str(text)
    text = re.sub(r'[\u200b\u200c\u200d\ufeff]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    return text

def clean_portuguese_text(text):

    text = clean_text(text)
    text = re.sub(r'\s*\([^)]*\)\s*', ' ', text)
    
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    text = re.sub(r',\s*,', ',', text)
    text = re.sub(r',\s*$', '', text)
    
    return text

df['Português'] = df['Português'].apply(clean_portuguese_text)
df['Tupi Antigo'] = df['Tupi Antigo'].apply(clean_text)
df = df[(df['Português'] != '') & (df['Tupi Antigo'] != '')]

df = df.drop(columns=['len_pt', 'len_ta'], errors='ignore')

print(f"Após limpeza: {len(df)} frases")


Após limpeza: 7097 frases


### 2.3 Divisão do Corpus

Dividi corpus em três conjuntos:
- **Treino (70%)**: Para fine-tuning do modelo
- **Validação (15%)**: Para early stopping e seleção de hiperparâmetros
- **Teste (15%)**: Para avaliação final (nunca usado durante treinamento)

In [11]:
from sklearn.model_selection import train_test_split

np.random.seed(CONFIG["seed"])
train_df, temp_df = train_test_split(df, test_size=0.30, random_state=CONFIG["seed"])
val_df, test_df = train_test_split(temp_df, test_size=0.50, random_state=CONFIG["seed"])

print(f"Divisão do corpus:")
print(f"  Treino:     {len(train_df)} ({len(train_df)/len(df)*100:.1f}%)")
print(f"  Validação:  {len(val_df)} ({len(val_df)/len(df)*100:.1f}%)")
print(f"  Teste:      {len(test_df)} ({len(test_df)/len(df)*100:.1f}%)")
print(f"  Total:      {len(df)}")

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

os.makedirs("./data", exist_ok=True)

train_df.to_csv("./data/train.csv", index=False)
val_df.to_csv("./data/val.csv", index=False)
test_df.to_csv("./data/test.csv", index=False)

print("Subconjuntos salvos em ./data/")

Divisão do corpus:
  Treino:     4967 (70.0%)
  Validação:  1065 (15.0%)
  Teste:      1065 (15.0%)
  Total:      7097
Subconjuntos salvos em ./data/


In [12]:
def df_to_dataset(df):
    return Dataset.from_dict({
        "portuguese": df["Português"].tolist(),
        "tupi": df["Tupi Antigo"].tolist()
    })

dataset_dict = DatasetDict({
    "train": df_to_dataset(train_df),
    "validation": df_to_dataset(val_df),
    "test": df_to_dataset(test_df)
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['portuguese', 'tupi'],
        num_rows: 4967
    })
    validation: Dataset({
        features: ['portuguese', 'tupi'],
        num_rows: 1065
    })
    test: Dataset({
        features: ['portuguese', 'tupi'],
        num_rows: 1065
    })
})


## 3. Modelo Tokenizador

In [13]:
tokenizer = MBart50TokenizerFast.from_pretrained(CONFIG["model_checkpoint"])

LANG_CODE = "pt_XX"

PREFIX_PT_TO_TA = "traduzir para tupi: "
PREFIX_TA_TO_PT = "traduzir para português: "

In [14]:
def preprocess_pt_to_ta(examples):

    inputs = examples["portuguese"]
    targets = examples["tupi"]
    
    tokenizer.src_lang = LANG_CODE
    tokenizer.tgt_lang = LANG_CODE
    
    model_inputs = tokenizer(
        inputs, 
        max_length=CONFIG["max_input_length"], 
        truncation=True,
        padding="max_length"
    )
    
    labels = tokenizer(
        text_target=targets,
        max_length=CONFIG["max_target_length"],
        truncation=True,
        padding="max_length"
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def preprocess_ta_to_pt(examples):
    """Pré-processa para tradução Tupi Antigo -> Português."""
    inputs = examples["tupi"]
    targets = examples["portuguese"]
    
    tokenizer.src_lang = LANG_CODE
    tokenizer.tgt_lang = LANG_CODE
    
    model_inputs = tokenizer(
        inputs, 
        max_length=CONFIG["max_input_length"], 
        truncation=True,
        padding="max_length"
    )
    
    labels = tokenizer(
        text_target=targets,
        max_length=CONFIG["max_target_length"],
        truncation=True,
        padding="max_length"
    )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

## 4. Configuração das Métricas de Avaliação

Métricas implementadas conforme especificado no enunciado

In [15]:

bleu_metric = evaluate.load("sacrebleu")
chrf_metric = evaluate.load("chrf")

def compute_all_metrics(predictions, references):

    refs_for_bleu = [[ref] for ref in references]
    bleu_result = bleu_metric.compute(predictions=predictions, references=refs_for_bleu)
    
    chrf1_result = chrf_metric.compute(
        predictions=predictions, 
        references=refs_for_bleu,
        char_order=6,
        word_order=0,
        beta=1
    )

    chrf3_result = chrf_metric.compute(
        predictions=predictions, 
        references=refs_for_bleu,
        char_order=6,
        word_order=0,
        beta=3
    )
    
    return {
        "bleu": bleu_result["score"],
        "chrf1": chrf1_result["score"],
        "chrf3": chrf3_result["score"],
        "bleu_details": {
            "precisions": bleu_result["precisions"],
            "brevity_penalty": bleu_result["bp"],
            "length_ratio": bleu_result["sys_len"] / bleu_result["ref_len"] if bleu_result["ref_len"] > 0 else 0
        }
    }

---
## 5. Zero-Shot


### Usar NLLB com Guarani como Proxy

Conforme orientação do professor:
- O **mBART com pt→pt** produz tradução quase nula no zero-shot
- O modelo **NLLB-200** possui suporte ao **Guarani** (`grn_Latn`), língua da família Tupi-Guarani relacionada ao Tupi Antigo
- Usamos Guarani como língua proxy para aproximar o Tupi Antigo

### Transformação Guarani → Tupi Antigo

Para melhorar a correspondência entre a saída do modelo (Guarani) e o Tupi Antigo de referência, apliquei transformações básicas conforme sugerido pelo professor:
- `ñ` → `nh` (nasalização)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

NLLB_CHECKPOINT = "facebook/nllb-200-distilled-600M"

print("Carregando modelo NLLB-200 para zero-shot...")
print("(NLLB possui Guarani - língua da família Tupi-Guarani)")

tokenizer_nllb = AutoTokenizer.from_pretrained(NLLB_CHECKPOINT)
model_zero_shot = AutoModelForSeq2SeqLM.from_pretrained(
    NLLB_CHECKPOINT,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
model_zero_shot = model_zero_shot.to(device)
model_zero_shot.eval()

LANG_PT_NLLB = "por_Latn"
LANG_GN_NLLB = "grn_Latn"

In [ ]:
def translate_batch_nllb(model, tokenizer, texts, source_lang, target_lang, batch_size=8, 
                          num_beams=5, max_length=128):
    model.eval()
    translations = []
    
    tokenizer.src_lang = source_lang
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        
        inputs = tokenizer(
            batch, 
            return_tensors="pt", 
            padding=True, 
            truncation=True,
            max_length=max_length
        ).to(device)
        
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                forced_bos_token_id=tokenizer.convert_tokens_to_ids(target_lang),
                max_length=max_length,
                num_beams=num_beams,
                early_stopping=True
            )
        
        decoded = tokenizer.batch_decode(generated, skip_special_tokens=True)
        translations.extend(decoded)
        
        if (i // batch_size + 1) % 20 == 0:
            print(f"  Traduzidos: {min(i + batch_size, len(texts))}/{len(texts)}")
    
    return translations

def guarani_to_tupi_transform(text):

    text = text.replace('ñ', 'nh')
    text = text.replace('Ñ', 'Nh')
    return text

In [ ]:
def translate_batch(model, texts, source_lang, target_lang, batch_size=8):
    model.eval()
    translations = []
    
    tokenizer.src_lang = source_lang
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        
        inputs = tokenizer(
            batch, 
            return_tensors="pt", 
            padding=True, 
            truncation=True,
            max_length=CONFIG["max_input_length"]
        ).to(device)
        
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                forced_bos_token_id=tokenizer.lang_code_to_id[target_lang],
                max_length=CONFIG["max_target_length"],
                num_beams=5,
                early_stopping=True
            )
        
        decoded = tokenizer.batch_decode(generated, skip_special_tokens=True)
        translations.extend(decoded)
        
        if (i // batch_size + 1) % 10 == 0:
            print(f"  Traduzidos: {min(i + batch_size, len(texts))}/{len(texts)}")
    
    return translations

In [ ]:
test_portuguese = test_df["Português"].tolist()
test_tupi_ref = test_df["Tupi Antigo"].tolist()
test_tupi = test_df["Tupi Antigo"].tolist()
test_portuguese_ref = test_df["Português"].tolist()

### 5.1 Zero-Shot: Português para Tupi Antigo (via Guarani)

In [ ]:
print("=" * 70)
print("ZERO-SHOT: Português → Tupi Antigo (via NLLB + Guarani)")
print("=" * 70)

print("\nTraduzindo PT → Guarani...")
translations_pt_gn = translate_batch_nllb(
    model_zero_shot,
    tokenizer_nllb,
    test_portuguese,
    source_lang=LANG_PT_NLLB,
    target_lang=LANG_GN_NLLB,
    num_beams=5,
    max_length=CONFIG["max_target_length"]
)

print("Aplicando transformação Guarani → Tupi Antigo (ñ → nh)...")
translations_pt_ta_zero = [guarani_to_tupi_transform(t) for t in translations_pt_gn]

metrics_pt_ta_zero = compute_all_metrics(translations_pt_ta_zero, test_tupi_ref)

print(f"\nTraduzidas {len(translations_pt_ta_zero)} frases.")
print("\n=== Métricas Zero-Shot PT → TA (via Guarani) ===")
print(f"  BLEU:  {metrics_pt_ta_zero['bleu']:.2f}")
print(f"  chrF1: {metrics_pt_ta_zero['chrf1']:.2f}")
print(f"  chrF3: {metrics_pt_ta_zero['chrf3']:.2f}")

### 5.2 Zero-Shot: Tupi Antigo para Português (via Guarani)

In [ ]:
print("=" * 70)
print("ZERO-SHOT: Tupi Antigo → Português (via NLLB + Guarani)")
print("=" * 70)

print("\nTraduzindo Tupi (como Guarani) → PT...")
translations_ta_pt_zero = translate_batch_nllb(
    model_zero_shot,
    tokenizer_nllb,
    test_tupi,
    source_lang=LANG_GN_NLLB,
    target_lang=LANG_PT_NLLB,
    num_beams=5,
    max_length=CONFIG["max_target_length"]
)

metrics_ta_pt_zero = compute_all_metrics(translations_ta_pt_zero, test_portuguese_ref)

print(f"\nTraduzidas {len(translations_ta_pt_zero)} frases.")
print("\n=== Métricas Zero-Shot TA → PT (via Guarani) ===")
print(f"  BLEU:  {metrics_ta_pt_zero['bleu']:.2f}")
print(f"  chrF1: {metrics_ta_pt_zero['chrf1']:.2f}")
print(f"  chrF3: {metrics_ta_pt_zero['chrf3']:.2f}")

### 5.3 Salva Resultados Zero-Shot

In [ ]:
results_zero_shot = {
    "pt_to_ta": {
        "bleu": metrics_pt_ta_zero["bleu"],
        "chrf1": metrics_pt_ta_zero["chrf1"],
        "chrf3": metrics_pt_ta_zero["chrf3"],
        "bleu_details": metrics_pt_ta_zero["bleu_details"]
    },
    "ta_to_pt": {
        "bleu": metrics_ta_pt_zero["bleu"],
        "chrf1": metrics_ta_pt_zero["chrf1"],
        "chrf3": metrics_ta_pt_zero["chrf3"],
        "bleu_details": metrics_ta_pt_zero["bleu_details"]
    }
}

with open(os.path.join(CONFIG["results_dir"], "results_zero_shot.json"), "w", encoding="utf-8") as f:
    json.dump(results_zero_shot, f, indent=2, ensure_ascii=False)

os.makedirs(os.path.join(CONFIG["results_dir"], "outputs_zero_shot"), exist_ok=True)

pd.DataFrame({
    "source": test_portuguese,
    "reference": test_tupi_ref,
    "translation": translations_pt_ta_zero
}).to_csv(os.path.join(CONFIG["results_dir"], "outputs_zero_shot", "pt_to_ta.csv"), index=False)

pd.DataFrame({
    "source": test_tupi,
    "reference": test_portuguese_ref,
    "translation": translations_ta_pt_zero
}).to_csv(os.path.join(CONFIG["results_dir"], "outputs_zero_shot", "ta_to_pt.csv"), index=False)

In [ ]:
# Liberar memória do modelo zero-shot
del model_zero_shot
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("Memória liberada.")

---

## 6. Fine-Tuning (Few-Shot Learning)

In [ ]:
def create_trainer(model, tokenizer, train_dataset, val_dataset, output_dir, direction):

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    
    def compute_metrics_trainer(eval_preds):
        preds, labels = eval_preds
        
        if isinstance(preds, tuple):
            preds = preds[0]
        
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        
        refs = [[l] for l in decoded_labels]
        bleu_result = bleu_metric.compute(predictions=decoded_preds, references=refs)
        
        return {"bleu": bleu_result["score"]}
    
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        learning_rate=CONFIG["learning_rate"],
        per_device_train_batch_size=CONFIG["batch_size"],
        per_device_eval_batch_size=CONFIG["batch_size"],
        num_train_epochs=CONFIG["num_epochs"],
        weight_decay=CONFIG["weight_decay"],
        warmup_ratio=CONFIG["warmup_ratio"],
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="bleu",
        greater_is_better=True,
        predict_with_generate=True,
        generation_max_length=CONFIG["max_target_length"],
        fp16=torch.cuda.is_available(),
        logging_dir=f"{output_dir}/logs",
        logging_steps=50,
        save_total_limit=2,
        report_to="none"
    )
    
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        processing_class=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics_trainer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=CONFIG["early_stopping_patience"])]
    )
    
    return trainer

### 6.1 Fine-Tuning: Português para Tupi Antigo

In [ ]:
tokenized_train_pt_ta = dataset_dict["train"].map(preprocess_pt_to_ta, batched=True)
tokenized_val_pt_ta = dataset_dict["validation"].map(preprocess_pt_to_ta, batched=True)

In [ ]:
model_pt_ta = MBartForConditionalGeneration.from_pretrained(
    CONFIG["model_checkpoint"],
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

lora_config = LoraConfig(
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    target_modules=["q_proj", "v_proj"],
    lora_dropout=CONFIG["lora_dropout"],
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model_pt_ta = get_peft_model(model_pt_ta, lora_config)
model_pt_ta.print_trainable_parameters()

In [ ]:
print("Treinamento PT → TA...")
trainer_pt_ta = create_trainer(
    model=model_pt_ta,
    tokenizer=tokenizer,
    train_dataset=tokenized_train_pt_ta,
    val_dataset=tokenized_val_pt_ta,
    output_dir=os.path.join(CONFIG["models_dir"], "pt_to_ta"),
    direction="pt_to_ta"
)

train_result_pt_ta = trainer_pt_ta.train()

print("\n=== Treinamento PT → TA Concluído ===")
print(f"Épocas: {train_result_pt_ta.metrics.get('epoch', 'N/A')}")
print(f"Loss final: {train_result_pt_ta.metrics.get('train_loss', 'N/A'):.4f}")

In [ ]:
model_pt_ta.save_pretrained(os.path.join(CONFIG["models_dir"], "pt_to_ta_final"))
tokenizer.save_pretrained(os.path.join(CONFIG["models_dir"], "pt_to_ta_final"))

In [ ]:
#Libera memoria do modelo PT -> TA
del model_pt_ta, trainer_pt_ta
if torch.cuda.is_available():
    torch.cuda.empty_cache()

### 6.2 Fine-Tuning: Tupi Antigo para Português

In [ ]:
tokenized_train_ta_pt = dataset_dict["train"].map(preprocess_ta_to_pt, batched=True)
tokenized_val_ta_pt = dataset_dict["validation"].map(preprocess_ta_to_pt, batched=True)

In [ ]:
model_ta_pt = MBartForConditionalGeneration.from_pretrained(
    CONFIG["model_checkpoint"],
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)

model_ta_pt = get_peft_model(model_ta_pt, lora_config)
model_ta_pt.print_trainable_parameters()

In [ ]:
print("Treinamento TA → PT...")
trainer_ta_pt = create_trainer(
    model=model_ta_pt,
    tokenizer=tokenizer,
    train_dataset=tokenized_train_ta_pt,
    val_dataset=tokenized_val_ta_pt,
    output_dir=os.path.join(CONFIG["models_dir"], "ta_to_pt"),
    direction="ta_to_pt"
)

train_result_ta_pt = trainer_ta_pt.train()

print("\n=== Treinamento TA → PT Concluído ===")
print(f"Épocas: {train_result_ta_pt.metrics.get('epoch', 'N/A')}")
print(f"Loss final: {train_result_ta_pt.metrics.get('train_loss', 'N/A'):.4f}")

In [ ]:
model_ta_pt.save_pretrained(os.path.join(CONFIG["models_dir"], "ta_to_pt_final"))
tokenizer.save_pretrained(os.path.join(CONFIG["models_dir"], "ta_to_pt_final"))

---

## 7. Avaliação Fine-Tuned

In [ ]:
from peft import PeftModel

def load_finetuned_model(model_path):
    """Carrega um modelo fine-tuned com LoRA."""
    base_model = MBartForConditionalGeneration.from_pretrained(
        CONFIG["model_checkpoint"],
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    )
    model = PeftModel.from_pretrained(base_model, model_path)
    model = model.to(device)
    model.eval()
    return model

model_pt_ta_ft = load_finetuned_model(os.path.join(CONFIG["models_dir"], "pt_to_ta_final"))
model_ta_pt_ft = load_finetuned_model(os.path.join(CONFIG["models_dir"], "ta_to_pt_final"))

### 7.1 Avaliação PT → TA (Fine-Tuned)

In [ ]:
print("Traduzindo PT → TA (fine-tuned)...")
translations_pt_ta_ft = translate_batch(
    model_pt_ta_ft,
    test_portuguese,
    source_lang=LANG_CODE,
    target_lang=LANG_CODE
)

metrics_pt_ta_ft = compute_all_metrics(translations_pt_ta_ft, test_tupi_ref)

print("\n=== Métricas Fine-Tuned PT → TA ===")
print(f"  BLEU:  {metrics_pt_ta_ft['bleu']:.2f}")
print(f"  chrF1: {metrics_pt_ta_ft['chrf1']:.2f}")
print(f"  chrF3: {metrics_pt_ta_ft['chrf3']:.2f}")

### 7.2 Avaliação TA → PT (Fine-Tuned)

In [ ]:
print("Traduzindo TA → PT (fine-tuned)...")
translations_ta_pt_ft = translate_batch(
    model_ta_pt_ft,
    test_tupi,
    source_lang=LANG_CODE,
    target_lang=LANG_CODE
)

metrics_ta_pt_ft = compute_all_metrics(translations_ta_pt_ft, test_portuguese_ref)

print("\n=== Métricas Fine-Tuned TA → PT ===")
print(f"  BLEU:  {metrics_ta_pt_ft['bleu']:.2f}")
print(f"  chrF1: {metrics_ta_pt_ft['chrf1']:.2f}")
print(f"  chrF3: {metrics_ta_pt_ft['chrf3']:.2f}")

In [ ]:
results_few_shot = {
    "pt_to_ta": {
        "bleu": metrics_pt_ta_ft["bleu"],
        "chrf1": metrics_pt_ta_ft["chrf1"],
        "chrf3": metrics_pt_ta_ft["chrf3"],
        "bleu_details": metrics_pt_ta_ft["bleu_details"]
    },
    "ta_to_pt": {
        "bleu": metrics_ta_pt_ft["bleu"],
        "chrf1": metrics_ta_pt_ft["chrf1"],
        "chrf3": metrics_ta_pt_ft["chrf3"],
        "bleu_details": metrics_ta_pt_ft["bleu_details"]
    }
}

with open(os.path.join(CONFIG["results_dir"], "results_few_shot.json"), "w", encoding="utf-8") as f:
    json.dump(results_few_shot, f, indent=2, ensure_ascii=False)

os.makedirs(os.path.join(CONFIG["results_dir"], "outputs_few_shot"), exist_ok=True)

pd.DataFrame({
    "source": test_portuguese,
    "reference": test_tupi_ref,
    "translation": translations_pt_ta_ft
}).to_csv(os.path.join(CONFIG["results_dir"], "outputs_few_shot", "pt_to_ta.csv"), index=False)

pd.DataFrame({
    "source": test_tupi,
    "reference": test_portuguese_ref,
    "translation": translations_ta_pt_ft
}).to_csv(os.path.join(CONFIG["results_dir"], "outputs_few_shot", "ta_to_pt.csv"), index=False)

---

## 8. Comparação: Zero-Shot vs Fine-Tuned

### Matriz Comparativa de Métricas

In [ ]:
comparison_data = {
    "Direção": ["PT → TA", "PT → TA", "TA → PT", "TA → PT"],
    "Modo": ["Zero-Shot", "Fine-Tuned", "Zero-Shot", "Fine-Tuned"],
    "BLEU": [
        metrics_pt_ta_zero["bleu"],
        metrics_pt_ta_ft["bleu"],
        metrics_ta_pt_zero["bleu"],
        metrics_ta_pt_ft["bleu"]
    ],
    "chrF1": [
        metrics_pt_ta_zero["chrf1"],
        metrics_pt_ta_ft["chrf1"],
        metrics_ta_pt_zero["chrf1"],
        metrics_ta_pt_ft["chrf1"]
    ],
    "chrF3": [
        metrics_pt_ta_zero["chrf3"],
        metrics_pt_ta_ft["chrf3"],
        metrics_ta_pt_zero["chrf3"],
        metrics_ta_pt_ft["chrf3"]
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("=== Matriz Comparativa de Métricas ===\n")
print(comparison_df.to_string(index=False))

# Calcular melhorias
print("\n\n=== Melhoria com Fine-Tuning ===")
print(f"\nPT → TA:")
print(f"  BLEU:  {metrics_pt_ta_ft['bleu'] - metrics_pt_ta_zero['bleu']:+.2f} pontos")
print(f"  chrF1: {metrics_pt_ta_ft['chrf1'] - metrics_pt_ta_zero['chrf1']:+.2f} pontos")
print(f"  chrF3: {metrics_pt_ta_ft['chrf3'] - metrics_pt_ta_zero['chrf3']:+.2f} pontos")

print(f"\nTA → PT:")
print(f"  BLEU:  {metrics_ta_pt_ft['bleu'] - metrics_ta_pt_zero['bleu']:+.2f} pontos")
print(f"  chrF1: {metrics_ta_pt_ft['chrf1'] - metrics_ta_pt_zero['chrf1']:+.2f} pontos")
print(f"  chrF3: {metrics_ta_pt_ft['chrf3'] - metrics_ta_pt_zero['chrf3']:+.2f} pontos")

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    metrics_names = ["BLEU", "chrF1", "chrF3"]
    x = np.arange(2)
    width = 0.35
    
    for idx, metric in enumerate(metrics_names):
        ax = axes[idx]
        zero_shot = [metrics_pt_ta_zero[metric.lower()], metrics_ta_pt_zero[metric.lower()]]
        fine_tuned = [metrics_pt_ta_ft[metric.lower()], metrics_ta_pt_ft[metric.lower()]]
        
        bars1 = ax.bar(x - width/2, zero_shot, width, label='Zero-Shot', color='#ff7f0e')
        bars2 = ax.bar(x + width/2, fine_tuned, width, label='Fine-Tuned', color='#1f77b4')
        
        ax.set_xlabel('Direção')
        ax.set_ylabel('Score')
        ax.set_title(f'{metric}')
        ax.set_xticks(x)
        ax.set_xticklabels(['PT → TA', 'TA → PT'])
        ax.legend()
        ax.set_ylim(0, 100)
        
        for bar in bars1 + bars2:
            height = bar.get_height()
            ax.annotate(f'{height:.1f}',
                       xy=(bar.get_x() + bar.get_width() / 2, height),
                       xytext=(0, 3),
                       textcoords="offset points",
                       ha='center', va='bottom', fontsize=8)
    
    plt.suptitle('Comparação: Zero-Shot vs Fine-Tuned', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(CONFIG["results_dir"], "comparison_chart.png"), dpi=150, bbox_inches='tight')
    plt.show()
    print(f"\nGráfico salvo em {CONFIG['results_dir']}/comparison_chart.png")
except ImportError:
    print("matplotlib não disponível para visualização gráfica")